# Module 2:
Make API call
- Collect data from API &amp; store in csv
- Scrape static websites &amp; store in csv
---
Scenario
---

1. You have been given a API with api documentation.
2. Make api call from python & store the resp in a python dict
2. (2A) *Optional*: Store the resp dict into a file (json)

3. Parse the following info from the API Response.

Example: URL 

https://min-api.cryptocompare.com/data/price?fsym=USD&tsyms=JPY,INR
```
{
"JPY": 107.93,
"INR": 84.82
}
```
4. Store that parsed infomation/data into a csv file.

| sno | from_symbol | to_symbol | price | datetime |
| ---  | ---  | ---  | ---  | --- |
| 1 | USD | INR | 71 | ... |
| 2 | USD | SGD | 1.37| ... |

use python `time` or `datetime` module to get the current time, when you make api call and store that into the csv file..


In [0]:
# Code here

In [30]:
import requests
from datetime import datetime
import pandas as pd

def get_data_from_api(fsym, tsym):
    url = f"https://min-api.cryptocompare.com/data/price?fsym={fsym}&tsyms={tsym}"
    resp = requests.get(url)
    data = resp.json()
    
    tsyms_list = tsym.split(",")
    
    records = []
    for tsym_item in tsyms_list:
        record = {
            "from_symbol": fsym,
            "to_symbol": tsym_item,
            "price": data[tsym_item],
            "datetime": str(datetime.now())
        }
        records.append(record)
    
    return records

def save_records_to_csv(records, fname="price.csv",mode="a",header=False,index=False):
    df = pd.DataFrame(records)
    df.to_csv(fname, mode=mode, index=index,header=header)

In [31]:
records = get_data_from_api('USD', 'JPY,INR')
save_records_to_csv(records,mode="w", header=True)

## Part 2: Scrape static websites


https://www.exchange-rates.org/converter/USD/EUR/1


Have a look at the html as well, to parse the exchange rate.

![https://i.imgur.com/z87bNYj.png](https://i.imgur.com/z87bNYj.png)

1. Look at the URL `path parameter` and decide how to pass inputs..
2. Use `bs4` library to parse the html as show below.
3. You have to extract the price shown here and save it into a csv file..

| sno | from_symbol | to_symbol | price | datetime |
| ---  | ---  | ---  | ---  | --- |
| 1 | USD | EUR | 0.9017| ... |

--- 


In [34]:
from bs4 import BeautifulSoup

def get_data_from_website(fsym, tsym):
    url = f"https://www.exchange-rates.org/converter/{fsym}/{tsym}/1"
    resp = requests.get(url)
    data = resp.text
    soup =BeautifulSoup(data)
    
    spantag = soup.find("span", {"id": "ctl00_M_lblToAmount"})
    if spantag:
        price = float(spantag.text)
    else:
        price = 'na'
        
    records = [
        {
            "from_symbol": fsym,
            "to_symbol": tsym,
            "price": price,
            "datetime": str(datetime.now())
        }
    ]
    return records


records = get_data_from_website("USD","EUR")
#save_records_to_csv(records)
save_records_to_csv(records,mode="a", header=False)

## Optional

** Try this if you completed the base workshop **
---
1. Store the same data into mongodb as well
  - create a `exchange` db in your mlab account.
  - create a collection `exchange-rates`
  - You can store sample record which looks like the following json.

```
{
    "from_symbol": "USD",
    'to_symbol' : "SGD",
    "price" : 1.37,
    "datetime" : "...."
}
```

## END